# "ETL" Yelp JSON to Pandas DataFrame
Ryan Skinner, December 2, 2018

***

The Yelp dataset contains 57,273 records of restaurants. We have already performed a support and distribution analysis of business attributes.

This notebook reads the raw JSON and adds all restaurant business records, in addition to their attributes that have high support based on prior analysis, to a Pandas DataFrame. This will facilitate combining features with other team members' and integrating with scikit-learn to build predictive models.

In [19]:
import pandas as pd
import json
import re
import string
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from pprint import pprint
from progressbar import ProgressBar

Because the Yelp dataset is so large, we cannot load it all into memory at once. To keep memory usage manageable, we read it into a pandas dataframe in chunks, select only those records that are restaurants, and then merge them with our main dataframe. In the process, we assign values to binary, nominal, and ordinal attributes.

In [20]:
def cast_vals(string):
    '''Casts values in a Yelp business attribute string that appear as text to the appropriate type.

    Casts performed:
    - True/False >> boolean
    - Numbers >> float or int as appropriate
    - Else keep as string
    '''
    if 'True' in string:
        return True
    if 'False' in string:
        return False
    if string.replace('.','').isdigit():
        if '.' in string:
            return float(string)
        else:
            return int(string)
    return string

In [21]:
def lower_and_keep_alphanum(s):
    '''Converts string to lowercase and strips all but alphanumeric characters, then appends it to str1'''
    
    pattern = re.compile('[\W_]+')
    return pattern.sub('', s.lower())

In [22]:
def code_business_categories(category_list):
    '''Given a list of Yelp business category strings, convert to onehot keys and values.
    Note: ignores business categories that do not have support > 0.02 out of all restaurants in Yelp dataset.'''

    categories_meeting_support_threshold = [
        'Food', 'Nightlife', 'Bars', 'Sandwiches', 'Fast Food',
        'American (Traditional)', 'Pizza', 'Burgers', 'Breakfast & Brunch', 'Italian',
        'Mexican', 'Chinese', 'American (New)', 'Coffee & Tea', 'Cafes',
        'Japanese', 'Chicken Wings', 'Seafood', 'Salad', 'Event Planning & Services',
        'Sushi Bars', 'Delis', 'Canadian (New)', 'Asian Fusion', 'Mediterranean',
        'Barbeque', 'Sports Bars', 'Specialty Food', 'Caterers', 'Steakhouses',
        'Desserts', 'Bakeries', 'Indian', 'Thai', 'Pubs',
        'Diners', 'Vietnamese', 'Middle Eastern']
    
    new_columns = []
    for category in category_list:
        if category in categories_meeting_support_threshold:
            new_columns.append('bizcategory_' + lower_and_keep_alphanum(category))
    new_values = [True] * len(new_columns)
    
    return new_columns, new_values

In [23]:
def code_business_attributes(attribute_dict):
    '''Given a dictionary representing Yelp business attributes, convert to onehot keys and values.
    Note: ignores business attributes that do not have support > 0.59 out of all restaurants in Yelp dataset.'''
    
    attributes_meeting_support_threshold = [
        'GoodForMeal', 'RestaurantsTakeOut', 'RestaurantsPriceRange2', 'RestaurantsGoodForGroups',
        'GoodForKids', 'RestaurantsReservations', 'RestaurantsDelivery', 'BusinessAcceptsCreditCards',
        'RestaurantsAttire', 'OutdoorSeating', 'HasTV', 'Alcohol', 'RestaurantsTableService',
        'NoiseLevel', 'WiFi', 'BikeParking', 'Caters'] #, 'BusinessParking', 'Ambience', 'WheelchairAccessible']
    
    categorical_to_ordinal = {'casual': 0, 'dressy': 1, 'formal': 2,
                              'none': 0, 'beer_and_wine': 1, 'full_bar': 2,
                              'quiet': 0, 'average': 1, 'loud': 2, 'very_loud': 3,
                              'no': False, 'free': True, 'paid': True} # Treating WiFi as binary

    # Compiling the patterns speeds up sub-category matching slightly in this case.
    # We are trying to extract keys and values from the Yelp dictionary format, which looks like...
    #  {'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}
    pattern_keys = re.compile('\'(.+?)\'')
    pattern_vals = re.compile(': (.+?)[,}]')
    
    try:
        new_columns = []
        new_values = []
        # Loop over items in business attribute dictionary
        for key, val_str in attribute_dict.items():
            if key in attributes_meeting_support_threshold:
                if val_str[0] == '{':
                    # This attribute has nested sub-categories, each with values
                    subkeys = pattern_keys.findall(val_str)
                    subvals = pattern_vals.findall(val_str)
                    for subkey, subval in zip(subkeys, subvals):
                        new_columns.append('bizattribute_' +
                                           lower_and_keep_alphanum(key) + '_' +
                                           lower_and_keep_alphanum(subkey))
                        new_values.append(cast_vals(subval))
                else:
                    # This attribute has a single value
                    new_columns.append('bizattribute_' + lower_and_keep_alphanum(key))
                    tmp_val = cast_vals(val_str)
                    if tmp_val in categorical_to_ordinal.keys():
                        tmp_val = categorical_to_ordinal[tmp_val]
                    new_values.append(tmp_val)
    except AttributeError as e:
        # Error caused by business not having attributes; just ignore
        return [], []

    return new_columns, new_values

In [24]:
#import timeit
#start_time = timeit.default_timer()

df_chunked = pd.read_json('/home/ryan/Yelp_Dataset/original_dataset/yelp_academic_dataset_business.json',
                  orient='records', lines='true', chunksize=1e4)

# The main dataframe where all restaurant records will end up
df_main = pd.DataFrame()

pbar = ProgressBar(max_value=57173)
nrestaurants = 0
for df in pbar(df_chunked):
    
    # The intermediary dataframe to which we will append restaurants before joining with the main dataframe
    df_chunk = pd.DataFrame()
    
    # Iterate over rows; one row equals one restaurant
    for rowindex, rowval in df.iterrows():
        
        cats = rowval.categories
        attrs = rowval.attributes
        try:
            cats = [item.strip() for item in cats.split(',')]
        except AttributeError as e:
            # Error caused by business not having categories; ignore this business
            continue
        if not 'Restaurants' in cats:
            # Further ignore businesses that are not restaurants
            continue
        # If we make it this far, this business is a restaurant!
        nrestaurants += 1
        
        # Start building the columns and values to be stored in the main dataframe for this restaurant
        biz_columns = ['address', 'business_id', 'city', 'hours', 'is_open',
                       'latitude', 'longitude', 'name', 'neighborhood', 'postal_code',
                       'review_count', 'stars', 'state']
        biz_values = rowval[biz_columns].tolist()
        
        # Process and add columns and values corresponding to business categories
        new_columns, new_values = code_business_categories(cats)
        biz_columns += new_columns
        biz_values += new_values
        
        # Process and add columns and values corresponding to business attributes
        new_columns, new_values = code_business_attributes(attrs)
        biz_columns += new_columns
        biz_values += new_values
        
        #pprint(list(zip(biz_columns, biz_values)))
        
        # Add this restaurant to the intermediate chunk dataframe
        row = pd.Series(data=biz_values, index=biz_columns)
        df_chunk = df_chunk.append(row, ignore_index=True)
    
        pbar.update(nrestaurants)
    
    # Append the intermediate chunk dataframe to the main dataframe
    df_main = df_main.append(df_chunk, ignore_index=True)

# Finally, set the index of the main restaurants dataframe
df_main.set_index('business_id', drop=True, inplace=True)

#elapsed = timeit.default_timer() - start_time
#elapsed

100% (57273 of 57273) |###################| Elapsed Time: 0:18:38 Time: 0:18:38


I conducted some very minimal timing tests to eke out a little more performance. It turns out that by only growing the large main dataframe intermittently, as opposed to adding each restaurant to the main dataframe ASAP, we can save time. Obviously we are loading 57,273 restaurants instead of 509, but the chunk size will be larger for the actual runs as well, so we hope to maintain some gains.

```
chunksize 100, nrestaurants 509
appending individual rows to main dataframe: 6.92 sec, 6.64 sec, 6.48 sec
appending  chunks of rows to main dataframe: 5.26 sec, 5.32 sec, 5.17 sec
```

Reading the full set of restaurants into the dataframe takes around 20min on my laptop (while plugged in to power). I could potentially make this faster yet by optimizing the chunk size through further tests, but this process will not need to be repeated once I have the main dataframe in memory, so it's not worth making this step as efficient as possible.

The size is only about 73 MB. Note below that `deep=True` interrogating object dtypes to accurately assess system-level memory consumption. This is more accurate (and reports a considerably higher value) in our case, below.

In [25]:
print('Size of df_main (deep=True)  {:.2f} MB'.format(df_main.memory_usage(index=True, deep=True).sum() / 1e6))
print('Size of df_main (deep=False) {:.2f} MB'.format(df_main.memory_usage(index=True, deep=False).sum() / 1e6))

Size of df_main (deep=True)  72.60 MB
Size of df_main (deep=False) 33.39 MB


Including features with support > 0.4, we would have had 87 "core" features that comprise this dataframe. However, per the discussion below, we have excluded the three feature classes with lowest support: BusinessParking, Ambience, and WheelchairAccessible. This brings our support threshold up to about 0.6, and the number of "core" features down to 72.

In [26]:
df_main.shape

(57173, 72)

The attribute counts (below) are all fairly high.

In [27]:
descrip = df_main.describe().transpose()

In [28]:
descrip.sort_values(by='count', ascending=False)

,count,mean,std,min,25%,50%,75%,max
stars,57173.0,3.444930,0.803564,1.000000,3.000000,3.500000,4.000000,5.000000
is_open,57173.0,0.723104,0.447469,0.000000,0.000000,1.000000,1.000000,1.000000
latitude,57173.0,40.154664,5.373212,-71.753941,35.269544,41.323585,43.713748,59.438181
review_count,57173.0,63.924108,162.644692,3.000000,7.000000,19.000000,59.000000,7968.000000
longitude,57173.0,-91.837977,18.234148,-123.587426,-112.064753,-81.333106,-79.422309,115.086769
bizattribute_restaurantstakeout,51142.0,0.936197,0.244404,0.000000,1.000000,1.000000,1.000000,1.000000
bizattribute_restaurantspricerange2,51111.0,1.676117,0.625056,1.000000,1.000000,2.000000,2.000000,4.000000
bizattribute_restaurantsgoodforgroups,50231.0,0.863929,0.342868,0.000000,1.000000,1.000000,1.000000,1.000000
bizattribute_goodforkids,49863.0,0.833624,0.372421,0.000000,1.000000,1.000000,1.000000,1.000000
bizattribute_restaurantsreservations,49537.0,0.391849,0.488168,0.000000,0.000000,0.000000,1.000000,1.000000


Some attributes with support between 0.4 and 0.6 also had mean values that are very close to zero (in particular, BusinessParking, Ambience, and WheelchairAccessible, which we already removed from the dataset). These had low enough support to be worth excluding. In particular, note any business attributes with less than 5% support.

- We excluded 'Ambiance' because it has low support anyways and has many values, further reducing the support of each value. Including 'Ambiance' could easily allow a model to overfit, especially for a single city.

- We also removed all 'BusinessParking' values for the same reason, especially the 'validated' value because its support was so low. Others may have been useful, like 'valet', though that one could just be a proxy for restaurant price range.

Note our actual value of `min_supp` is 0.59, based on the following table:
```
                            count   support
GoodForMeal                 53199  0.930492
RestaurantsTakeOut          51142  0.894513
RestaurantsPriceRange2      51111  0.893971
RestaurantsGoodForGroups    50231  0.878579
GoodForKids                 49863  0.872142
RestaurantsReservations     49537  0.866440
RestaurantsDelivery         48900  0.855299
BusinessAcceptsCreditCards  48451  0.847445
RestaurantsAttire           48164  0.842426
OutdoorSeating              47552  0.831721
HasTV                       44343  0.775593
Alcohol                     43786  0.765851
RestaurantsTableService     43313  0.757578
NoiseLevel                  40896  0.715303
WiFi                        39861  0.697200
BikeParking                 38143  0.667151
Caters                      34180  0.597835
------ categories below are excluded ------
BusinessParking             32594  0.570094
Ambience                    28158  0.492505
WheelchairAccessible        23729  0.415039```

In [31]:
descrip.loc[descrip['mean'] != 1].sort_values(by='mean', ascending=True)

,count,mean,std,min,25%,50%,75%,max
longitude,57173.0,-91.837977,18.234148,-123.587426,-112.064753,-81.333106,-79.422309,115.086769
bizattribute_restaurantsattire,48164.0,0.033594,0.194160,0.000000,0.000000,0.000000,0.000000,2.000000
bizattribute_goodformeal_dessert,46925.0,0.048652,0.215142,0.000000,0.000000,0.000000,0.000000,1.000000
bizattribute_goodformeal_latenight,46925.0,0.057091,0.232019,0.000000,0.000000,0.000000,0.000000,1.000000
bizattribute_goodformeal_brunch,46925.0,0.090123,0.286360,0.000000,0.000000,0.000000,0.000000,1.000000
bizattribute_goodformeal_breakfast,46925.0,0.090719,0.287212,0.000000,0.000000,0.000000,0.000000,1.000000
bizattribute_restaurantsdelivery,48900.0,0.256830,0.436889,0.000000,0.000000,0.000000,1.000000,1.000000
bizattribute_outdoorseating,47552.0,0.389300,0.487597,0.000000,0.000000,0.000000,1.000000,1.000000
bizattribute_goodformeal_dinner,46925.0,0.390325,0.487828,0.000000,0.000000,0.000000,1.000000,1.000000
bizattribute_restaurantsreservations,49537.0,0.391849,0.488168,0.000000,0.000000,0.000000,1.000000,1.000000


Now we save the restaurants DataFrame to memory as a pickle so we don't need to wait 20+ minutes each time we want to access it...

In [32]:
df_main.to_pickle('restaurants_vanilla.pkl.bz2') #  3.3 MB
#df_main.to_pickle('restaurants_vanilla.pkl')    # 47.0 MB